In [1]:
%cd ..

C:\Users\hp\dev\video-membership-app


In [2]:
from app import db
from app.config import get_settings
from app.playlist.models import Playlist
from app.videos.models import Video
from cassandra.cqlengine.management import sync_table

In [3]:
session = db.get_session()
settings = get_settings()
allow_table_drop = True

In [4]:
if allow_table_drop:
    session.execute(f"DROP TABLE IF EXISTS {settings.keyspace}.playlist")

In [5]:
sync_table(Playlist)
sync_table(Video)

In [6]:
obj = Playlist.objects.create(title="Hello Video")
obj

Playlist(db_id=UUID('31780f9f-e744-11ed-91fb-c46516f58b27'), user_id=None, updated=datetime.datetime(2023, 4, 30, 10, 45, 49, 315670, tzinfo=datetime.timezone.utc), host_ids=[], title='Hello Video')

In [7]:
from datetime import datetime

obj.updated = datetime.utcnow()
obj.save()

Playlist(db_id=UUID('31780f9f-e744-11ed-91fb-c46516f58b27'), user_id=None, updated=datetime.datetime(2023, 4, 30, 10, 45, 59, 281531), host_ids=[], title='Hello Video')

In [8]:
obj.host_id = ['some new list']
obj.save()
obj

Playlist(db_id=UUID('31780f9f-e744-11ed-91fb-c46516f58b27'), user_id=None, updated=datetime.datetime(2023, 4, 30, 10, 45, 59, 281531), host_ids=[], title='Hello Video')

In [14]:
list(Playlist.objects.all())

[Playlist(db_id=UUID('31780f9f-e744-11ed-91fb-c46516f58b27'), user_id=None, updated=datetime.datetime(2023, 4, 30, 10, 45, 59, 491000), host_ids=['X3qUfOB3x8Q', 'RDE6Uz73A7g', '9ix7TUGVYIo', 'itnqEauWQZM', 'nNpvWBuTfrc'], title='Hello Video')]

In [9]:
video_q = list(Video.objects.filter(host_id__in=obj.host_ids))
video_q

[]

In [10]:
host_ids = list(Video.objects.all().limit(5).values_list("host_id", flat=True))
host_ids

['X3qUfOB3x8Q', 'RDE6Uz73A7g', '9ix7TUGVYIo', 'itnqEauWQZM', 'nNpvWBuTfrc']

In [11]:
obj.add_host_ids(host_ids=host_ids)

True

In [12]:
video_q = list(Video.objects.filter(host_id__in=obj.host_ids))
video_q

[Video(title=None, host_id=9ix7TUGVYIo, host_service=youtube),
 Video(title=The Hunger Games: The Ballad of Songbirds & Snakes (2023) Official Trailer, host_id=RDE6Uz73A7g, host_service=youtube),
 Video(title=VS Code Day 2023, host_id=X3qUfOB3x8Q, host_service=youtube),
 Video(title=Transformers: Rise of the Beasts | Official Trailer (2023 Movie), host_id=itnqEauWQZM, host_service=youtube),
 Video(title=None, host_id=nNpvWBuTfrc, host_service=youtube)]

In [15]:
obj.get_videos()

[Video(title=VS Code Day 2023, host_id=X3qUfOB3x8Q, host_service=youtube),
 Video(title=The Hunger Games: The Ballad of Songbirds & Snakes (2023) Official Trailer, host_id=RDE6Uz73A7g, host_service=youtube),
 Video(title=None, host_id=9ix7TUGVYIo, host_service=youtube),
 Video(title=Transformers: Rise of the Beasts | Official Trailer (2023 Movie), host_id=itnqEauWQZM, host_service=youtube),
 Video(title=None, host_id=nNpvWBuTfrc, host_service=youtube)]